In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql import functions as func
from functools import reduce


spark = SparkSession.builder \
 .appName("moving_average")\
 .config("Pausedspark.sql.shuffle.partitions", "4") \
 .getOrCreate()


In [2]:
  df = spark.read \
 .option("inferSchema", "true") \
 .option("header", "true") \
 .option("delimiter", ",") \
 .csv("/home/spark/Downloads/archive/rejected_2007_to_2018q4.csv")

In [3]:
## Schema print just after loading file. so it has space in column name
df.printSchema()

root
 |-- Amount Requested: double (nullable = true)
 |-- Application Date: string (nullable = true)
 |-- Loan Title: string (nullable = true)
 |-- Risk_Score: string (nullable = true)
 |-- Debt-To-Income Ratio: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Employment Length: string (nullable = true)
 |-- Policy Code: string (nullable = true)



In [4]:
## Total count of file
df.count()


27648741

In [5]:

current_columns = df.columns # Returns list of columns as python list
print(current_columns)

new_columns = list(map(lambda item : item.replace(" ","_").upper(),current_columns)) 

# Replacing white spaces with '_' and converting the whole column name to upper case

print(new_columns)


df_ma100 = reduce(lambda data, idx: data.withColumnRenamed(current_columns[idx], new_columns[idx]), range(len(current_columns)), df)

df_ma100.printSchema()



['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code']
['AMOUNT_REQUESTED', 'APPLICATION_DATE', 'LOAN_TITLE', 'RISK_SCORE', 'DEBT-TO-INCOME_RATIO', 'ZIP_CODE', 'STATE', 'EMPLOYMENT_LENGTH', 'POLICY_CODE']
root
 |-- AMOUNT_REQUESTED: double (nullable = true)
 |-- APPLICATION_DATE: string (nullable = true)
 |-- LOAN_TITLE: string (nullable = true)
 |-- RISK_SCORE: string (nullable = true)
 |-- DEBT-TO-INCOME_RATIO: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- EMPLOYMENT_LENGTH: string (nullable = true)
 |-- POLICY_CODE: string (nullable = true)



In [11]:
## Filterd only year 2008 amd 2009 records
df_ma100=df_ma100.filter((year("APPLICATION_DATE") > lit("2007")) & (year("APPLICATION_DATE") <= lit("2009")))

## count of 2007 data only
df_ma100.count()

82587

In [12]:
##converting timestamp column to seconds, and then using use the rangeBetween function in the pyspark.sql.Window class to include the correct rows in window
df_ma100 = df_ma100.withColumn('APPLICATION_DATE', df_ma100.APPLICATION_DATE.cast('timestamp'))
days = lambda i: i * 86400
df_ma100 = df_ma100.withColumn('APPLICATION_DATE', df_ma100.APPLICATION_DATE.cast('timestamp'))
windowSpec = Window.orderBy(func.col("APPLICATION_DATE").cast('long')).rangeBetween(-days(50), 0)
df2 = df_ma100.withColumn('RiskScoreMA50', func.avg("RISK_SCORE").over(windowSpec)) 

## select only few columns
df2[to_date('APPLICATION_DATE').alias('APPLICATION_DATE'),'RISK_SCORE','RiskScoreMA50'].show(100,False)

+----------------+----------+-----------------+
|APPLICATION_DATE|RISK_SCORE|RiskScoreMA50    |
+----------------+----------+-----------------+
|2008-01-01      |null      |522.8888888888889|
|2008-01-01      |511.0     |522.8888888888889|
|2008-01-01      |null      |522.8888888888889|
|2008-01-01      |0.0       |522.8888888888889|
|2008-01-01      |680.0     |522.8888888888889|
|2008-01-01      |500.0     |522.8888888888889|
|2008-01-01      |685.0     |522.8888888888889|
|2008-01-01      |536.0     |522.8888888888889|
|2008-01-01      |661.0     |522.8888888888889|
|2008-01-01      |671.0     |522.8888888888889|
|2008-01-01      |684.0     |522.8888888888889|
|2008-01-01      |746.0     |522.8888888888889|
|2008-01-01      |670.0     |522.8888888888889|
|2008-01-01      |592.0     |522.8888888888889|
|2008-01-01      |630.0     |522.8888888888889|
|2008-01-01      |612.0     |522.8888888888889|
|2008-01-01      |557.0     |522.8888888888889|
|2008-01-01      |485.0     |522.8888888

In [13]:
## Inserted the data in hive table.
df2.write.mode("append").saveAsTable("honest.rejected_loan_2008_2009")

In [16]:
## Created temp table on top of the dataframe
df2.createOrReplaceTempView("rejected_loan_2008_2009") 

## just a fitler check to show we have data of of 2007 year only
spark.sql("select count(*) from rejected_loan_2008_2009 where year(APPLICATION_DATE) not in ('2008','2009')").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [17]:
## selecting all the column from table;
spark.sql("select * from rejected_loan_2008_2009").show()

+----------------+-------------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+-----------------+
|AMOUNT_REQUESTED|   APPLICATION_DATE|          LOAN_TITLE|RISK_SCORE|DEBT-TO-INCOME_RATIO|ZIP_CODE|STATE|EMPLOYMENT_LENGTH|POLICY_CODE|    RiskScoreMA50|
+----------------+-------------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+-----------------+
|         20000.0|2008-01-01 00:00:00|  debt_consolidation|      null|                100%|   281xx|   NC|         < 1 year|        0.0|522.8888888888889|
|          2000.0|2008-01-01 00:00:00|      virgin islands|     511.0|               4.93%|   008xx|   FL|          4 years|        0.0|522.8888888888889|
|         20000.0|2008-01-01 00:00:00|  debt_consolidation|      null|                100%|   920xx|   CA|        10+ years|        0.0|522.8888888888889|
|         15000.0|2008-01-01 00:00:00|      major_purchase|       0.0|